# **LP3_Run3**

In [11]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

ERROR: Operation cancelled by user
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached glpk-0.4.7.tar.gz (161 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for glpk (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for glpk
Failed to build glpk
ERROR: Could not build wheels for glpk, which is required to install pyproject.toml-based projects


In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits."""

In [6]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [7]:
print(response1.choices[0].message.content)

Let's denote the number of units produced for each product by the company as: P1, P2, P3, where P1 is the number of units of product P1 produced, and so on.

As the company also wish to know how much to subcontract the smelting operation for products P1 and P2, the number of units to be subcontracted for smelting would be SP1 and SP2 for the respective products. 

Therefore, our decision variables are:
1) P1: Number of units of product P1 produced by Proda, S.A
2) P2: Number of units of product P2 produced by Proda, S.A
3) P3: Number of units of product P3 produced by Proda, S.A
4) SP1: Number of units of product P1 subcontracting for smelting
5) SP2: Number of units of product P2 subcontracting for smelting


In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [9]:
print(response2.choices[0].message.content)

The objective function to maximize weekly profits would then be:

max Z = 1.50*(P1 + SP1) - (0.30*P1 + 0.50*SP1 + 0.20*P1 + 0.30*P1) 
         + 1.80*(P2 + SP2) - (0.50*P2 + 0.60*SP2 + 0.10*P2 + 0.20*P2)
         + 1.97*P3 - (0.40*P3 + 0.27*P3 + 0.20*P3) 

Z to be maximized indicates the total weekly profit.


In [10]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [12]:
print(response3.choices[0].message.content)

Constraints on the operations time capacity are:

6*P1 + 10*P2 + 8*P3 <= 8000 (constraint on smelting time)

6*P1 + 3*P2 + 8*P3 <= 12000 (constraint on mechanization time)

3*P1 + 2*P2 + 2*P3 <= 10000 (constraint on assembly & packaging time)

Also, the amounts subcontracted for products P1 and P2 cannot exceed the production of these products:

SP1 <= P1
SP2 <= P2

Finally, all decision variables must be greater than or equal to zero:

P1, P2, P3, SP1, SP2 >= 0


### **Generate Pyomo Code**

In [13]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [14]:
Markdown(response4.choices[0].message.content)

Here's how you can translate the above problem into code using the Pyomo package for Python:

```python
# Import the Pyomo library
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define the decision variables, lower bound is 0
model.P1 = Var(within=NonNegativeReals)
model.P2 = Var(within=NonNegativeReals)
model.P3 = Var(within=NonNegativeReals)
model.SP1 = Var(within=NonNegativeReals)
model.SP2 = Var(within=NonNegativeReals)

# Define the objective function
model.profit = Objective(expr = 1.50*(model.P1 + model.SP1) - (0.30*model.P1 + 0.50*model.SP1 + 0.20*model.P1 + 0.30*model.P1) 
                         + 1.80*(model.P2 + model.SP2) - (0.50*model.P2 + 0.60*model.SP2 + 0.10*model.P2 + 0.20*model.P2)
                         + 1.97*model.P3 - (0.40*model.P3 + 0.27*model.P3 + 0.20*model.P3),
                         sense=maximize)

# Define the constraints
model.constr_smelting = Constraint(expr = 6*model.P1 + 10*model.P2 + 8*model.P3 <= 8000)
model.constr_mechanization = Constraint(expr = 6*model.P1 + 3*model.P2 + 8*model.P3 <= 12000)
model.constr_assembly_packaging = Constraint(expr = 3*model.P1 + 2*model.P2 + 2*model.P3 <= 10000)
model.constr_SP1 = Constraint(expr = model.SP1 <= model.P1)
model.constr_SP2 = Constraint(expr = model.SP2 <= model.P2)

# Solve the problem
solver = SolverFactory('glpk')
solver.solve(model)

# Print solution to the console
print("P1 = ", model.P1())
print("P2 = ", model.P2())
print("P3 = ", model.P3())
print("SP1 = ", model.SP1())
print("SP2 = ", model.SP2())
```
This code defines a Pyomo model, sets the constraints based on production capacities, uses the GLPK solver to find the solution, and prints the results to the console. 

Please note that solving process requires `glpk` solver installed in your system. You could install it from command line using `sudo apt-get install glpk-utils`. Or if it can't be installed, you could use other solvers like `'gurobi'`, `'cplex'` etc. if installed.

### **Run the code Generated by GPT4**

In [16]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define the decision variables, lower bound is 0
model.P1 = Var(within=NonNegativeReals)
model.P2 = Var(within=NonNegativeReals)
model.P3 = Var(within=NonNegativeReals)
model.SP1 = Var(within=NonNegativeReals)
model.SP2 = Var(within=NonNegativeReals)

# Define the objective function
model.profit = Objective(expr = 1.50*(model.P1 + model.SP1) - (0.30*model.P1 + 0.50*model.SP1 + 0.20*model.P1 + 0.30*model.P1)
                         + 1.80*(model.P2 + model.SP2) - (0.50*model.P2 + 0.60*model.SP2 + 0.10*model.P2 + 0.20*model.P2)
                         + 1.97*model.P3 - (0.40*model.P3 + 0.27*model.P3 + 0.20*model.P3),
                         sense=maximize)

# Define the constraints
model.constr_smelting = Constraint(expr = 6*model.P1 + 10*model.P2 + 8*model.P3 <= 8000)
model.constr_mechanization = Constraint(expr = 6*model.P1 + 3*model.P2 + 8*model.P3 <= 12000)
model.constr_assembly_packaging = Constraint(expr = 3*model.P1 + 2*model.P2 + 2*model.P3 <= 10000)
model.constr_SP1 = Constraint(expr = model.SP1 <= model.P1)
model.constr_SP2 = Constraint(expr = model.SP2 <= model.P2)

# Solve the problem
solver = SolverFactory('glpk')
solver.solve(model)

# Print solution to the console
print("P1 = ", model.P1())
print("P2 = ", model.P2())
print("P3 = ", model.P3())
print("SP1 = ", model.SP1())
print("SP2 = ", model.SP2())
print(model.profit())

P1 =  1333.33333333333
P2 =  0.0
P3 =  0.0
SP1 =  1333.33333333333
SP2 =  0.0
2266.6666666666606


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**